## Etapa atual
- **Validação da ideia**: criação de MVP manual para comparação de preços.  
- **Criação de base inicial**: coletar dados de algumas notas fiscais ou produtos.  
- **Objetivo desta etapa**: ter **uma base inicial de preços** para análises e comparativos.

##  Estrutura para etapa atual
1. Criar **script para raspar dados das notas fiscais**:  
   - Extrair: produto, preço, mercado, data.  
2. Alimentar **planilha ou banco de dados simples** (CSV / Google Sheets / SQLite).  
3. Fazer **comparativo de preços iniciais** usando esses dados.  
4. Validar se o MVP manual fornece resultados confiáveis antes de automatizar scraping de sites ou QR Codes.


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [15]:
url = "https://www.nfce.fazenda.sp.gov.br/NFCeConsultaPublica/Paginas/ConsultaQRCode.aspx?p=35250944914620000101651070000018431107018733|2|1|1|6D1F127B092ACFC5A9130DD51D27B2DA25F4B2A7"

response = requests.get(url)

html = response.text

soup = BeautifulSoup(html, 'html.parser')

In [93]:
def cadastro_nome_produto(dados):
    tit_produto = dados.find_all('span', class_='txtTit')
    nomes_produtos = [p.get_text(strip=True) for p in tit_produto]
    return nomes_produtos

def cadastro_quantidade_comprada(dados):
    quantidade = dados.find_all('span', class_='Rqtd')
    quantidade_comprada = [p.get_text(strip=True).replace("Qtde.:", "") for p in quantidade]
    return quantidade_comprada

def cadastro_unidade_medida(dados):
    unidade = dados.find_all('span', class_='RUN')
    unidade_produto = [p.get_text(strip=True).replace("UN:", "") for p in unidade]
    return unidade_produto

def cadastro_valor_unitario(dados):
    valor_unitario = dados.find_all('span', class_='RvlUnit')
    vl_unitario = [
        p.get_text(strip=True).replace("Vl. Unit.:", "").replace(",", ".")
        for p in valor_unitario
    ]
    return vl_unitario

def cadastro_valor_total(dados):
    valor_total = dados.find_all('td', class_='txtTit noWrap')
    vl_total = [
        p.get_text(strip=True).replace("Vl. Total", "").replace(",", ".")
        for p in valor_total
    ]
    return vl_total

def cadastro_nome_mercado(dados):
    nome_empr = dados.find_all('div', class_='txtTopo')
    nome_empresa = [p.get_text(strip=True) for p in nome_empr]
    return nome_empresa

def cadastro_cnpj_empresa(dados): 
    documento = dados.find_all('div', class_='text')
    cnpjs = []
    for p in documento:
        texto = p.get_text(strip=True)
        if "CNPJ:" in texto:  # garante que só pega o bloco do CNPJ
            cnpjs.append(texto.replace("CNPJ:", "").strip())
    return cnpjs

def cadastro_data_emissao(dados):
    divs = dados.find_all('div', class_='text')
    for div in divs:
        texto = div.get_text(" ", strip=True)
        if "Emissão:" in texto:
            # exemplo: "Número: 1843 Série: 107 Emissão: 26/09/2025 19:46:00 - Via Consumidor"
            partes = texto.split("Emissão:")
            if len(partes) > 1:
                data_emissao = partes[1].replace("- Via Consumidor", "").strip()
                return data_emissao
    return None



def cria_dataframe(dados):
    # usa as funções que você já criou
    cnpj = cadastro_cnpj_empresa(dados)
    nome_empresa = cadastro_nome_mercado(dados)
    nomes_produtos = cadastro_nome_produto(dados)
    quantidade_comprada = cadastro_quantidade_comprada(dados)
    unidade_produto = cadastro_unidade_medida(dados)
    vl_unitario = cadastro_valor_unitario(dados)
    vl_total = cadastro_valor_total(dados)
    data_emissao = cadastro_data_emissao(dados)

    # garante que os dados "fixos" repitam para cada produto
    cnpj = [cnpj[0]] * len(nomes_produtos)
    nome_empresa = [nome_empresa[0]] * len(nomes_produtos)
    data_emissao = [data_emissao] * len(nomes_produtos)

    # monta dicionário
    dados_dicionario = {
        'CNPJ': cnpj,
        'Empresa': nome_empresa,
        'Data Emissão': data_emissao,
        'Produto': nomes_produtos,
        'Qtde': quantidade_comprada,
        'UN': unidade_produto,
        'Vl. Unit.': vl_unitario,
        'Vl. Total': vl_total
    }

    # cria dataframe
    df = pd.DataFrame(dados_dicionario)

    # converte valores numéricos
    df['Qtde'] = df['Qtde'].str.replace(",", ".").astype(float)
    df['Vl. Unit.'] = df['Vl. Unit.'].astype(float)
    df['Vl. Total'] = df['Vl. Total'].astype(float)

    return df


In [104]:
divs = soup.find_all('strong', 'Emissão:')
for div in divs:
    print(div)

In [106]:
div

<div class="text">AV PRES. CASTELO BRANCO
							,
							2415
							,
							
							,
							JARDIM ZAIRA
							,
							MAUA
							,
							SP</div>